In [ ]:
from qiskit import *
from numpy import *
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_bloch_vector, array_to_latex
from qiskit.quantum_info import state_fidelity
import qiskit.quantum_info as qi
from qiskit.quantum_info import *
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import itertools

aer_sim = AerSimulator()


def eaqecc(permutation):
    qm= QuantumRegister(1,'q_main')
    qa= QuantumRegister(2,'q_ancilla')
    qe=QuantumRegister(2,'q_entangled')
    qs = QuantumRegister(4,'q_syndrome')
    creg= ClassicalRegister(4,'syndrome_bits')
    res= ClassicalRegister(1,"result")
    qc= QuantumCircuit(qs,qe,qa,qm,creg)
    qc.h(qe[0])
    qc.cx(qe[0],qe[1])
    initial_state = [sqrt(7)/3, sqrt(2)/3]
    sv1= Statevector(initial_state)                                        # Define state |q_0>
    qc.initialize(initial_state, qm)

    qc.x(qm) # this helps the state from being mixed 
    ##encoding circuit
    qc.h(qa[0])
    qc.h(qa[1])
    qc.cx(qa[1],qm)
    qc.cx(qa[0],qa[1])
    qc.h(qm)
    qc.s(qa[0])
    qc.h(qa[1])
    qc.cx(qa[0],qa[1])
    qc.cx(qa[0],qm)
    qc.h(qa[0])
    qc.cx(qe[1],qa[0])
    qc.cx(qe[1],qm)
    qc.h(qe[1])
    qc.h(qm)
    qc.cx(qe[1],qa[0])
    qc.cx(qe[1],qa[1])
    qc.h(qa[0])
    qc.h(qa[1])
    qc.swap(qe[1],qa[0])
    qc.barrier()

    positions = {'X': [], 'Y': [], 'Z': [], 'I':[]}
    for index, gate in enumerate(permutation):
    positions[gate].append(index)

    x=positions['X']
    y=positions['Y']
    z= positions['Z']
    i= positions['I']
    x_gate = [X + 5 for X in x]
    y_gate = [Y + 5 for Y in y]
    z_gate = [Z + 5 for Z in z]
    i_gate = [I + 5 for I in i]
    if len(x_gate)!=0:
        qc.x(x_gate)
    if len(y_gate)!=0:
        qc.y(y_gate)
    if len(z_gate)!=0:
        qc.z(z_gate)
    if len(i_gate)!=0:
        qc.id(i_gate)

    w=len(x)+len(y)+len(z)
    qc.barrier()
    # channel ends here

    qc.h(qs)
    qc.barrier()
    # ZIZZX
    qc.cx(qs[0],qe[0])
    qc.cz(qs[0],qe[1])
    qc.cz(qs[0],qa[0])
    qc.cz(qs[0],qm)

    #IZXZZ
    qc.cz(qs[1],qe[0])
    qc.cz(qs[1],qe[1])
    qc.cx(qs[1],qa[0])
    qc.cz(qs[1],qa[1])

    #XYYZI
    qc.cz(qs[2],qe[1])
    qc.cy(qs[2],qa[0])
    qc.cy(qs[2],qa[1])
    qc.cx(qs[2],qm)


    #-ZXXYI
    qc.cy(qs[3],qe[1])
    qc.cx(qs[3],qa[0])
    qc.cx(qs[3],qa[1])
    qc.cz(qs[3],qm)


    qc.barrier()
    qc.h(qs)
    qc.measure(qs,creg)
    #display(qc.draw())
    qc.barrier()
    # error correction circuit
    qc.x(qm).c_if(creg,1)
    qc.y(qm).c_if(creg,5)
    qc.z(qm).c_if(creg,12)

    qc.x(qa[1]).c_if(creg,14)
    qc.y(qa[1]).c_if(creg,2)
    qc.z(qa[1]).c_if(creg,4)

    qc.x(qa[0]).c_if(creg,13)
    qc.y(qa[0]).c_if(creg,3)
    qc.z(qa[0]).c_if(creg,6)

    qc.x(qe[1]).c_if(creg,7)
    qc.y(qe[1]).c_if(creg,15)
    qc.z(qe[1]).c_if(creg,0)

    # decoding circuit
    qc.barrier()
    qc.swap(qe[1],qa[0])
    qc.h(qa[1])
    qc.h(qa[0])
    qc.cx(qe[1],qa[1])
    qc.cx(qe[1],qa[0])
    qc.h(qm)
    qc.h(qe[1])
    qc.cx(qe[1],qm)
    qc.cx(qe[1],qa[0])
    qc.h(qa[0])
    qc.cx(qa[0],qm)
    qc.cx(qa[0],qa[1])
    qc.h(qa[1])
    qc.s(qa[0])
    qc.h(qm)
    qc.cx(qa[0],qa[1])
    qc.cx(qa[1],qm)
    qc.h(qa[1])
    qc.h(qa[0])
    # qc.measure(qm,res)
    qc.x(qm)
    qc.save_density_matrix(qubits=[8])
    sim_density = AerSimulator()
    job= sim_density.run(qc)
    result = job.result().data()
    dm= result.get('density_matrix')

    sv_f = dm.to_statevector()
    res= sv_f.equiv(sv1)
    return res,w

In [ ]:
places = 5

gates = ['X','I','Y','Z']
index =[]
actual_stabilier=[]

# Generate all permutations
permutations = itertools.product(gates, repeat=places)
for permutation in permutations:
    permutation_str = ''.join(permutation)
    result,n= five_qubit(permutation_str)
                        # Get the actual key from the dictionary
    if result==True:
        index.append(n)
        actual_stabilier.append(permutation_str)


        
        
def evaluate_expression(w_value):
    # Define the symbols
    p, w = sp.symbols('p w')

    # Define the expression
    expression = (1 - (3*p/4))**(5 - w) * (p/4)**w

    # Substitute the value of w into the expression
    evaluated_expression = expression.subs(w, w_value)
    simplified_expression = sp.expand(evaluated_expression)

    return simplified_expression

def sum_evaluated_expressions(w_values):
    # Define the symbol
    p = sp.symbols('p')

    # Initialize the total expression
    total_expression = 0

    # Sum the evaluated expressions for each w value
    for w_value in w_values:
        total_expression += evaluate_expression(w_value)

    return total_expression

# Example usage
w_values = index  # Example values for w
total_expr = sum_evaluated_expressions(w_values)
print(f"The total evaluated expression for w values is: {total_expr}")